<a href="https://colab.research.google.com/github/lipeng2021/-python1/blob/main/%E6%A8%A1%E5%9E%8B%E6%9E%84%E9%80%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import torch
import torch.nn as nn

In [30]:
class MLP(nn.Module):
  def __init__(self,**kwargs):
    super(MLP,self).__init__(**kwargs)
    self.hidden = nn.Linear(784,256)
    self.act = nn.ReLU()
    self.output = nn.Linear(256,10)
  def forward(self,x):
    y = self.output(self.act(self.hidden(x)))
    return y

In [31]:
x = torch.rand(2,784)
net = MLP()
print(net)
net(x)


MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.2782,  0.2191, -0.2027, -0.1833, -0.0266, -0.1501, -0.0025,  0.0933,
          0.0979, -0.1029],
        [ 0.4179,  0.1537, -0.1378, -0.1933,  0.0387, -0.1275,  0.0035,  0.0686,
          0.2108, -0.0524]], grad_fn=<AddmmBackward0>)

In [32]:
from collections import OrderedDict
class MySequential(nn.Module):
  from collections import OrderedDict
  def __init__(self,*args):
    super(MySequential,self).__init__()
    if len(args)==1 and isinstance(args[0],OrderedDict):
      for key,module in args[0].items():
        self.add_module(key,module)
    else:
      for idx,module in enumerate(args):
        self.add_module(str(idx),module)
  def forward(self,input):
    for module in self._modules.values():
      input = module(input)
    return input

In [33]:
net = MySequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,10),
)
print(net)
net(x)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.1650,  0.1123, -0.0918, -0.1199,  0.0848, -0.0162, -0.2380,  0.0709,
         -0.0514,  0.1964],
        [-0.2359,  0.1676, -0.1905, -0.0381,  0.1168,  0.0494, -0.1824, -0.1149,
         -0.1075,  0.1750]], grad_fn=<AddmmBackward0>)

In [34]:
net = torch.nn.Sequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,10), 
)
print(net)
net(x)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.2910,  0.0973, -0.0355, -0.2241, -0.0249,  0.0337,  0.0232,  0.1273,
          0.2002, -0.1346],
        [-0.2665,  0.0984,  0.0193, -0.2858,  0.0639, -0.0281,  0.0538,  0.2797,
          0.2202,  0.0750]], grad_fn=<AddmmBackward0>)

In [ ]:
net1 = nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
net1.append(nn.Linear(256,10))
print(net1)
print(net1[1])

In [45]:
net=nn.ModuleDict({
    'hidden':nn.Linear(784,256),
    'act':nn.ReLU(),
})
net['output'] = nn.Linear(256,10)
print(net)
print(net['hidden'])
print(net.output)

ModuleDict(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)
Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)


In [51]:
class FancyMLP(nn.Module):
  def __init__(self,**kwargs):
    super(FancyMLP,self).__init__(**kwargs)
    self.rand_weight = torch.rand((20,20),requires_grad=False)
    self.linear = nn.Linear(20,20)
  def forward(self,x):
    x = self.linear(x)
    x = nn.functional.relu(torch.mm(x,self.rand_weight)+1)
    x = self.linear(x)
    while x.norm().item()>1:
      x /=2
    if x.norm().item()<0.8:
      x*=10
    return x,x.sum()


In [53]:
x = torch.rand(2,20)
net = FancyMLP()
net(x)
print(net)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


In [58]:
class NestMLP(nn.Module):
  def __init__(self,**kwargs):
    super(NestMLP,self).__init__(**kwargs)
    self.net = nn.Sequential(nn.Linear(40,30),nn.ReLU())
  def forward(self,x):
    return self.net(x)
net = nn.Sequential(NestMLP(),nn.Linear(30,20),FancyMLP())
x = torch.rand(2,40)

print(net)
net(x)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


(tensor([[-1.6740,  0.5847, -1.5395, -0.1208, -0.3700,  0.0590,  0.1122,  0.5159,
          -0.3026,  1.2844,  0.5595, -0.0845,  1.6344,  0.3923, -1.3881, -0.1865,
          -0.5994, -0.5453,  0.1299,  0.2003],
         [-1.5554,  0.4698, -1.4552, -0.1188, -0.3590,  0.0773,  0.1139,  0.4525,
          -0.2728,  1.1555,  0.5744, -0.0367,  1.5081,  0.4419, -1.2907, -0.2371,
          -0.5950, -0.5715,  0.1381,  0.2344]], grad_fn=<MulBackward0>),
 tensor(-2.6643, grad_fn=<SumBackward0>))